In [4]:
from pylabnet.hardware.awg.tektronix_awg70k import Driver
from pylabnet.utils.logging.logger import LogClient
import pylabnet.network.client_server.tektronix_awg70k as tektronix_awg70k
from pylabnet.network.core.generic_server import GenericServer

import numpy as np
import matplotlib.pyplot as plt
from pyvisa import VisaIOError, ResourceManager

client = tektronix_awg70k.Client(
    host='192.168.50.113', 
    port=16489
)

rm = ResourceManager()
my_instrument = rm.open_resource('TCPIP0::192.168.50.207::INSTR')

In [19]:
client.query('BWAVeform:AMPLitude?')

'500.0000000000E-3\n'

In [7]:
client.stop()

In [ ]:
print(my_instrument.query('*TST?'))

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.